# EAS Testing - Recents Fling on Android

The goal of this experiment is to collect frame statistics while swiping up and down tabs of recently opened applications on a Nexus N5X running Android with an EAS kernel. This process is name **Recents Fling**. The Analysis phase will consist in comparing EAS with other schedulers, that is comparing *sched* governor with:

    - interactive
    - performance
    - powersave
    - ondemand
    
For this experiment it is recommended to open many applications so that we can swipe over more recently opened applications.

In [1]:
import logging
reload(logging)
log_fmt = '%(asctime)-9s %(levelname)-8s: %(message)s'
logging.basicConfig(format=log_fmt)

# Change to info once the notebook runs ok
logging.getLogger().setLevel(logging.INFO)

In [2]:
%pylab inline

import copy
import os
from time import sleep

# Support to access the remote target
import devlib
from env import TestEnv

from devlib.utils.android import adb_command

# Support for trace events analysis
from trace import Trace
from trace_analysis import TraceAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test Environment set up

In case more than one Android device are conencted to the host, you must specify the ID of the device you want to target in `my_target_conf`. Run `adb devices` on your host to get the ID.

In [3]:
# Setup a target configuration
my_target_conf = {
    
    # Target platform and board
    "platform"    : 'android',
    
    # Device ID
    # "device" : "00b1346f0878ccb1",
    
    # Define devlib modules to load
    "modules"     : [
        'cpufreq'       # enable CPUFreq support
    ],
}

In [4]:
my_tests_conf = {

    # Folder where all the results will be collected
    "results_dir" : "AndroidN5X_RecentsFling",

    # Platform configurations to test
    "confs" : [
        {
            "tag"            : "recentsfling",
            "flags"          : "ftrace",           # Enable FTrace events
            "sched_features" : "ENERGY_AWARE",     # enable EAS
        },
    ],
    
    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_load_avg_cpu",
            "cpu_frequency",
            "cpu_capacity"
         ],
         "buffsize" : 10 * 1024,
    },
    
    # Tools required by the experiments
    "tools"   : [ 'trace-cmd' ],
}

In [5]:
# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

2016-03-29 14:28:57,726 INFO    :         Target - Using base path: /home/derkling/Code/lisa
2016-03-29 14:28:57,727 INFO    :         Target - Loading custom (inline) target configuration
2016-03-29 14:28:57,728 INFO    :         Target - Loading custom (inline) test configuration
2016-03-29 14:28:57,728 INFO    :         Target - Devlib modules to load: ['cpufreq']
2016-03-29 14:28:57,729 INFO    :         Target - Connecting Android target [DEFAULT]
2016-03-29 14:29:04,296 INFO    :         Target - Initializing target workdir:
2016-03-29 14:29:04,297 INFO    :         Target -    /data/local/tmp/devlib-target
2016-03-29 14:29:06,729 INFO    :         Target - Topology:
2016-03-29 14:29:06,730 INFO    :         Target -    [[0]]
2016-03-29 14:29:07,710 INFO    :         FTrace - Enabled tracepoints:
2016-03-29 14:29:07,711 INFO    :         FTrace -   sched_switch
2016-03-29 14:29:07,712 INFO    :         FTrace -   sched_load_avg_cpu
2016-03-29 14:29:07,713 INFO    :         FTrace

# Support Functions

This set of support functions will help us running the benchmark using different CPUFreq governors.

In [6]:
def set_performance():
    target.cpufreq.set_all_governors('performance')

def set_powersave():
    target.cpufreq.set_all_governors('powersave')

def set_interactive():
    target.cpufreq.set_all_governors('interactive')

def set_sched():
    target.cpufreq.set_all_governors('sched')

def set_ondemand():
    target.cpufreq.set_all_governors('ondemand')
    
    for cpu in target.list_online_cpus():
        tunables = target.cpufreq.get_governor_tunables(cpu)
        target.cpufreq.set_governor_tunables(
            cpu,
            'ondemand',
            **{'sampling_rate' : tunables['sampling_rate_min']}
        )

In [7]:
# CPUFreq configurations to test
confs = {
    'performance' : {
        'label' : 'prf',
        'set' :  set_performance,
    },
#     'powersave' : {
#         'label' : 'pws',
#         'set' :  set_powersave,
#     },
     'interactive' : {
         'label' : 'int',
         'set' :  set_interactive,
     },
#     'sched' : {
#         'label' : 'sch',
#         'set' :  set_sched,
#     },
#     'ondemand' : {
#         'label' : 'odm',
#         'set' :  set_ondemand,
#     }
}

# The set of results for each comparison test
results = {}

In [8]:
def recentsfling_run(exp_dir):
    # Open Recents on the target device
    target.execute('input keyevent KEYCODE_APP_SWITCH')
    # Allow the activity to start
    sleep(5)
    # Reset framestats collection
    target.execute('dumpsys gfxinfo com.android.systemui reset')
    logging.info('Start Swiping Recents')
    for i in range(5):
        # Simulate two fast UP and DOWN swipes
        target.execute('input swipe 500 1700 500 500 100')
        target.execute('input swipe 500 500 500 1700 100')
        # Simulate two slow UP and DOWN swipes
        target.execute('input swipe 500 1700 500 500 1000')
        target.execute('input swipe 500 500 500 1700 1000')
    logging.info('Swiping Recents Completed')
    
    # Get frame stats
    framestats_file = os.path.join(exp_dir, "framestats.txt")
    adb_command(target.adb_name, 'shell dumpsys gfxinfo com.android.systemui > {}'.format(framestats_file))
    
    # Close Recents
    target.execute('input keyevent KEYCODE_BACK')

    return framestats_file

In [9]:
def experiment(governor, exp_dir):
    os.system('mkdir -p {}'.format(exp_dir));

    logging.info('------------------------')
    logging.info('Run workload using %s governor', governor)
    confs[governor]['set']()
    
    # Start FTrace
    te.ftrace.start()
      
    ### Run the benchmark ###
    framestats_file = recentsfling_run(exp_dir)
    
    # Stop FTrace
    te.ftrace.stop()    

    # Collect and keep track of the trace
    trace_file = os.path.join(exp_dir, 'trace.dat')
    te.ftrace.get_trace(trace_file)
    
    # Parse trace
    tr = Trace(te.platform, exp_dir,
               events=my_tests_conf['ftrace']['events'])
    ta = TraceAnalysis(tr, te.platform)
    
    # return all the experiment data
    return {
        'dir'             : exp_dir,
        'framestats_file' : framestats_file,
        'trace'           : trace_file,
        'ftrace'          : tr.ftrace,
        'ta'              : ta
    }

# Run experiments and collect traces

In [10]:
# Unlock device screen (assume no password required)
target.execute('input keyevent 82')

# Run the benchmark in all the configured governors
for governor in confs:
    test_dir = os.path.join(te.res_dir, governor)
    res = experiment(governor, test_dir)
    results[governor] = copy.deepcopy(res)

2016-03-29 14:29:08,945 INFO    : ------------------------
2016-03-29 14:29:08,946 INFO    : Run workload using performance governor
2016-03-29 14:29:15,906 INFO    : Start Swiping Recents
2016-03-29 14:29:40,818 INFO    : Swiping Recents Completed
2016-03-29 14:29:46,433 INFO    : Parsing FTrace format...
2016-03-29 14:29:57,584 INFO    : Collected events spans a 31.842 [s] time interval
2016-03-29 14:29:57,584 INFO    : Set plots time range to (0.000000, 31.841528)[s]
2016-03-29 14:29:58,992 INFO    : ------------------------
2016-03-29 14:29:58,993 INFO    : Run workload using interactive governor
2016-03-29 14:30:06,735 INFO    : Start Swiping Recents
2016-03-29 14:30:32,688 INFO    : Swiping Recents Completed
2016-03-29 14:30:40,121 INFO    : Parsing FTrace format...
2016-03-29 14:30:50,118 INFO    : Collected events spans a 33.294 [s] time interval
2016-03-29 14:30:50,119 INFO    : Set plots time range to (0.000000, 33.293597)[s]


# UI Performance Analysis

In [11]:
for governor in confs:
    framestats_file = results[governor]['framestats_file']
    print "Frame Statistics for {} governor".format(governor.upper())
    !sed '/Stats since/,/99th/!d;/99th/q' $framestats_file
    print ""

Frame Statistics for PERFORMANCE governor
Stats since: 6499634704812ns
Total frames rendered: 1359
Janky frames: 144 (10.60%)
90th percentile: 17ms
95th percentile: 21ms
99th percentile: 46ms

Frame Statistics for INTERACTIVE governor
Stats since: 6550459445313ns
Total frames rendered: 1246
Janky frames: 228 (18.30%)
90th percentile: 24ms
95th percentile: 28ms
99th percentile: 40ms

